### Dekorator do wyznaczania czasu trwania wywołania funkcji  
https://towardsdatascience.com/12-python-decorators-to-take-your-code-to-the-next-level-a910a1ab3e99
https://towardsdatascience.com/10-fabulous-python-decorators-ab674a732871

In [ ]:
# funkcja do 

from functools import wraps
import time 

def timefn(fn):
    @wraps(fn)
    def measure_time(*args, **kwargs):
        t1 = time.time()
        result = fn(*args, **kwargs)
        t2 = time.time()
        print ("@timefn: " + fn.__name__ + " took " + str(t2 - t1) + " seconds")
        return result
    return measure_time

In [21]:
import random

if __name__ == "__main__":
    @timefn
    def losowa(x):
        for _ in range(x):
            a = random.random()
        return 

    losowa(10000000)

@timefn:losowa took 0.4927222728729248 seconds


### Dekorator dodający do funkci opcję Cashe, jeżeli nastąpi wywołanie dla tych samych argumentów to zamiast wykonać pełen kod zczytuje wartość z casha 
Dekorator może oczywiście byc podrasowany i pamięć będzie składana w pliku trwałym

In [23]:
import functools
def memoize(func):
    """
    Cache the results of the function so it doesn't need to be called
    again, if the same arguments are provided a second time.
    """
    cache = {}
    @functools.wraps(func)
    def wrapper(*args):
        if args in cache:
            return cache[args]
        # This line is for demonstration only.
        # Remove it before using it for real.
        print('Calling %s()' % func.__name__)
        result = func(*args)
        cache[args] = result
        return result
    return wrapper

In [26]:
if __name__ == "__main__":
    @memoize
    def silnia(n):
        if n==1:
            return 1
        return n*silnia(n-1)
    
    print(silnia(5))
    print("a za drugim razem mamy już tylko:")
    print(silnia(6))
    

Calling silnia()
Calling silnia()
Calling silnia()
Calling silnia()
Calling silnia()
120
a za drugim razem mamy już tylko:
Calling silnia()
720


### Sprawdzanie poprawności typów argumentów funkcji 
Jeżeli zależy nam aby argumenty jak i wartość zwracana przez funkcję był danego typu możemy zastosować następujący dekorator 

In [42]:
import inspect
import functools
from itertools import chain

def typesafe(func):
    """
    Verify that the function is called with the right argument types and
    that it returns a value of the right type, according to its annotations
    """
    spec = inspect.getfullargspec(func)
    annotations = spec.annotations
    for name, annotation in annotations.items():
        if not isinstance(annotation, type):
            raise TypeError("The annotation for '%s' is not a type." % name)
        error = "Wrong type for %s: expected %s, got %s."
        defaults = spec.defaults or ()
        defaults_zip = zip(spec.args[-len(defaults):], defaults)
        kwonlydefaults = spec.kwonlydefaults or {}
        for name, value in chain(defaults_zip, kwonlydefaults.items()):
            if name in annotations and not isinstance(value, annotations[name]):
                raise TypeError(error % ('default value of %s' % name,
                                        annotations[name].__name__,
                                        type(value).__name__))
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        # Populate a dictionary of explicit arguments passed positionally
        explicit_args = dict(zip(spec.args, args))
        keyword_args = kwargs.copy()
        # Add all explicit arguments passed by keyword
        for name in chain(spec.args, spec.kwonlyargs):
            if name in kwargs:
                explicit_args[name] = keyword_args.pop(name)
        # Deal with explicit arguments
        for name, arg in explicit_args.items():
            if name in annotations and not isinstance(arg, annotations[name]):
                raise TypeError(error % (name,
                                        annotations[name].__name__,
                                        type(arg).__name__))
        # Deal with variable positional arguments
        if spec.varargs and spec.varargs in annotations:
            annotation = annotations[spec.varargs]
            for i, arg in enumerate(args[len(spec.args):]):
                if not isinstance(arg, annotation):
                    raise TypeError(error % ('variable argument %s' % (i + 1),
                                            annotation.__name__,
                                            type(arg).__name__))
        # Deal with variable keyword arguments
        if spec.varkw and spec.varkw in annotations:
            annotation = annotations[spec.varkw]
            for name, arg in keyword_args.items():
                if not isinstance(arg, annotation):
                    raise TypeError(error % (name,
                                            annotation.__name__,
                                            type(arg).__name__))
        r = func(*args, **kwargs)
        if 'return' in annotations and not isinstance(r, annotations['return']):
            raise TypeError(error % ('the return value',
                                    annotations['return'].__name__,
                                    type(r).__name__))
        return r
    return wrapper

In [44]:
# przykładowe wywołanie które pownny wskazać błędy w wywołaniu obu funkcji

@typesafe
def prepend_rows(rows:list, prefix:str) -> list:
    return [prefix + row for row in rows]


aa = prepend_rows(rows=['112', 'tre'], prefix = 123)


TypeError: Wrong type for prefix: expected str, got int.

In [47]:
@typesafe
def prepend_rows(rows:list, prefix:str) -> list:
    return 1

prepend_rows(rows=['112', 'tre'], prefix = '123')

TypeError: Wrong type for the return value: expected list, got int.